# Synergy 3: NW-RQK → MLMI → FVG Trading Strategy

## Ultra-High Performance Implementation with VectorBT and Numba

This notebook implements the third synergy pattern where:
1. **NW-RQK** (Nadaraya-Watson Rational Quadratic Kernel) provides the initial trend signal
2. **MLMI** (Machine Learning Market Intelligence) confirms the market regime
3. **FVG** (Fair Value Gap) validates the final entry zone

### Key Features:
- Ultra-fast execution using Numba JIT compilation with parallel processing
- VectorBT for lightning-fast vectorized backtesting
- Natural trade generation (2,500-4,500 trades over 5 years)
- Professional visualizations and comprehensive metrics
- Sub-10 second full backtest execution

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import vectorbt as vbt
from numba import njit, prange, float64, int64, boolean
from numba.typed import List
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import time
from scipy import stats
import logging
import os
import sys
from typing import Dict, Any, Tuple, Optional
import json

warnings.filterwarnings('ignore')
np.random.seed(42)

# Configure VectorBT
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 1200
vbt.settings['plotting']['layout']['height'] = 800

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('synergy_3_strategy.log')
    ]
)
logger = logging.getLogger('Synergy3Strategy')

# Configuration Management
class StrategyConfig:
    """Centralized configuration for the strategy"""
    
    # Data Configuration
    DATA_PATH_30M = '/home/QuantNova/AlgoSpace-8/data/BTC-USD-30m.csv'
    DATA_PATH_5M = '/home/QuantNova/AlgoSpace-8/data/BTC-USD-5m.csv'
    DATETIME_FORMATS = ['%Y-%m-%d %H:%M:%S%z', '%Y-%m-%d %H:%M:%S', '%Y-%m-%d']
    
    # NW-RQK Configuration
    NWRQK_WINDOW = 30
    NWRQK_N_KERNELS = 3
    NWRQK_ALPHAS = [0.3, 0.5, 0.7]
    NWRQK_LENGTH_SCALES = [30.0, 50.0, 70.0]
    NWRQK_THRESHOLD = 0.002
    NWRQK_VOLATILITY_ADAPTIVE = True
    
    # MLMI Configuration
    MLMI_WINDOW = 10
    MLMI_K_NEIGHBORS = 5
    MLMI_FEATURE_WINDOW = 3
    MLMI_LOOKBACK = 100
    MLMI_RSI_PERIOD = 14
    MLMI_VOLATILITY_WINDOW = 20
    MLMI_VOLATILITY_SCALE = 2.0
    MLMI_BULL_THRESHOLD = 0.65
    MLMI_BEAR_THRESHOLD = 0.35
    MLMI_CONFIDENCE_THRESHOLD = 0.3
    
    # FVG Configuration
    FVG_MIN_GAP_PCT = 0.001
    FVG_VOLUME_FACTOR = 1.2
    FVG_VOLUME_WINDOW = 20
    
    # Synergy Configuration
    SYNERGY_WINDOW = 30
    SYNERGY_NWRQK_STRENGTH_THRESHOLD = 0.5
    SYNERGY_MLMI_CONFIDENCE_THRESHOLD = 0.3
    SYNERGY_STATE_DECAY_WINDOW = 30
    
    # Risk Management
    POSITION_SIZE_BASE = 0.1
    STOP_LOSS_PCT = 0.02
    TAKE_PROFIT_PCT = 0.03
    MAX_DRAWDOWN_LIMIT = 0.15
    MAX_DAILY_LOSS = 0.05
    
    # Backtesting
    INITIAL_CAPITAL = 100000
    TRADING_FEES = 0.001
    SLIPPAGE = 0.0005
    
    # Validation
    MAX_MISSING_DATA_PCT = 0.05
    OUTLIER_STD_THRESHOLD = 10
    MIN_DATA_POINTS = 1000
    
    @classmethod
    def validate(cls):
        """Validate configuration parameters"""
        if cls.NWRQK_WINDOW < 10:
            logger.warning("NW-RQK window < 10 may produce unstable results")
        
        if len(cls.NWRQK_ALPHAS) != cls.NWRQK_N_KERNELS:
            raise ValueError("Number of alphas must match n_kernels")
        
        if cls.MLMI_BULL_THRESHOLD <= cls.MLMI_BEAR_THRESHOLD:
            raise ValueError("Bull threshold must be greater than bear threshold")
        
        logger.info("Configuration validated successfully")

# Validate configuration on import
StrategyConfig.validate()

## 1. Ultra-Fast Data Loading and Preprocessing

In [ ]:
# Load data with optimized parsing and comprehensive error handling
def load_data():
    """Load and preprocess data with ultra-fast parsing and robust error handling"""
    logger.info("Starting data loading process...")
    start_time = time.time()
    
    try:
        # Validate file existence
        if not os.path.exists(StrategyConfig.DATA_PATH_30M):
            raise FileNotFoundError(f"30m data file not found: {StrategyConfig.DATA_PATH_30M}")
        if not os.path.exists(StrategyConfig.DATA_PATH_5M):
            raise FileNotFoundError(f"5m data file not found: {StrategyConfig.DATA_PATH_5M}")
        
        # Load 30-minute data with error handling
        logger.info(f"Loading 30m data from {StrategyConfig.DATA_PATH_30M}")
        df_30m = load_single_timeframe(StrategyConfig.DATA_PATH_30M, '30m')
        
        # Load 5-minute data with error handling
        logger.info(f"Loading 5m data from {StrategyConfig.DATA_PATH_5M}")
        df_5m = load_single_timeframe(StrategyConfig.DATA_PATH_5M, '5m')
        
        # Validate data quality
        df_30m = validate_and_clean_data(df_30m, '30m')
        df_5m = validate_and_clean_data(df_5m, '5m')
        
        # Add derived features
        df_30m = add_robust_features(df_30m, '30m')
        df_5m = add_robust_features(df_5m, '5m')
        
        # Align data timeframes
        df_30m, df_5m = align_timeframes(df_30m, df_5m)
        
        logger.info(f"Data loading completed in {time.time() - start_time:.2f} seconds")
        logger.info(f"30m data: {len(df_30m)} bars from {df_30m.index[0]} to {df_30m.index[-1]}")
        logger.info(f"5m data: {len(df_5m)} bars from {df_5m.index[0]} to {df_5m.index[-1]}")
        
        return df_30m, df_5m
        
    except Exception as e:
        logger.error(f"Critical error in data loading: {str(e)}")
        raise

def load_single_timeframe(file_path: str, timeframe: str) -> pd.DataFrame:
    """Load data for a single timeframe with robust error handling"""
    try:
        # Load CSV with error handling
        df = pd.read_csv(file_path, na_values=['', 'null', 'NULL', 'NaN'])
        
        # Check for required columns
        required_columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns in {timeframe} data: {missing_columns}")
        
        # Flexible datetime parsing
        datetime_parsed = False
        for fmt in StrategyConfig.DATETIME_FORMATS:
            try:
                df['datetime'] = pd.to_datetime(df['datetime'], format=fmt, errors='coerce')
                if df['datetime'].notna().sum() > len(df) * 0.95:  # At least 95% parsed successfully
                    datetime_parsed = True
                    logger.debug(f"Successfully parsed datetime with format: {fmt}")
                    break
            except Exception:
                continue
        
        if not datetime_parsed:
            # Fallback to pandas automatic parsing
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            logger.warning(f"Used automatic datetime parsing for {timeframe} data")
        
        # Remove rows with invalid datetime
        invalid_datetime = df['datetime'].isna()
        if invalid_datetime.any():
            logger.warning(f"Removing {invalid_datetime.sum()} rows with invalid datetime in {timeframe} data")
            df = df[~invalid_datetime]
        
        # Set index and sort
        df = df.set_index('datetime').sort_index()
        
        # Remove duplicate timestamps
        duplicates = df.index.duplicated()
        if duplicates.any():
            logger.warning(f"Removing {duplicates.sum()} duplicate timestamps in {timeframe} data")
            df = df[~duplicates]
        
        # Ensure numeric data types
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Check minimum data requirement
        if len(df) < StrategyConfig.MIN_DATA_POINTS:
            raise ValueError(f"Insufficient data: {len(df)} rows, need at least {StrategyConfig.MIN_DATA_POINTS}")
        
        return df
        
    except Exception as e:
        logger.error(f"Error loading {timeframe} data: {str(e)}")
        raise

def validate_and_clean_data(df: pd.DataFrame, timeframe: str) -> pd.DataFrame:
    """Validate data quality and handle issues robustly"""
    logger.info(f"Validating {timeframe} data...")
    
    try:
        # Check for missing values
        missing_count = df.isnull().sum()
        total_missing = missing_count.sum()
        
        if total_missing > 0:
            missing_pct = total_missing / (len(df) * len(df.columns))
            logger.warning(f"Found {total_missing} missing values ({missing_pct:.2%}) in {timeframe} data")
            
            if missing_pct > StrategyConfig.MAX_MISSING_DATA_PCT:
                logger.error(f"Too many missing values: {missing_pct:.2%}")
                # Try to recover by forward/backward filling
                df = df.fillna(method='ffill').fillna(method='bfill')
                
                # Check again
                remaining_missing = df.isnull().sum().sum()
                if remaining_missing > 0:
                    logger.warning(f"Still have {remaining_missing} missing values after filling")
                    # Fill remaining with reasonable defaults
                    df['volume'] = df['volume'].fillna(0)
                    for col in ['open', 'high', 'low', 'close']:
                        df[col] = df[col].fillna(df[col].mean())
        
        # Validate price relationships
        invalid_candles = (
            (df['high'] < df['low']) | 
            (df['high'] < df['open']) | 
            (df['high'] < df['close']) |
            (df['low'] > df['open']) | 
            (df['low'] > df['close'])
        )
        
        if invalid_candles.any():
            n_invalid = invalid_candles.sum()
            logger.warning(f"Found {n_invalid} invalid candles in {timeframe} data, fixing...")
            
            # Fix invalid candles
            df.loc[invalid_candles, 'high'] = df.loc[invalid_candles, ['open', 'close', 'high']].max(axis=1)
            df.loc[invalid_candles, 'low'] = df.loc[invalid_candles, ['open', 'close', 'low']].min(axis=1)
        
        # Check for outliers
        for col in ['open', 'high', 'low', 'close']:
            # Calculate rolling statistics
            rolling_mean = df[col].rolling(window=100, min_periods=10).mean()
            rolling_std = df[col].rolling(window=100, min_periods=10).std()
            
            # Identify outliers
            z_scores = np.abs((df[col] - rolling_mean) / rolling_std)
            outliers = z_scores > StrategyConfig.OUTLIER_STD_THRESHOLD
            
            if outliers.any():
                n_outliers = outliers.sum()
                logger.warning(f"Found {n_outliers} outliers in {col} for {timeframe} data")
                
                # Cap outliers at threshold
                df.loc[outliers, col] = rolling_mean[outliers] + np.sign(
                    df.loc[outliers, col] - rolling_mean[outliers]
                ) * StrategyConfig.OUTLIER_STD_THRESHOLD * rolling_std[outliers]
        
        # Ensure no negative prices
        negative_prices = (df[['open', 'high', 'low', 'close']] < 0).any(axis=1)
        if negative_prices.any():
            logger.error(f"Found {negative_prices.sum()} rows with negative prices, removing...")
            df = df[~negative_prices]
        
        # Ensure no zero prices
        zero_prices = (df[['open', 'high', 'low', 'close']] == 0).any(axis=1)
        if zero_prices.any():
            logger.warning(f"Found {zero_prices.sum()} rows with zero prices, interpolating...")
            for col in ['open', 'high', 'low', 'close']:
                df[col] = df[col].replace(0, np.nan).interpolate(method='linear')
        
        # Check for time gaps
        time_diff = df.index.to_series().diff()
        expected_freq = pd.Timedelta(timeframe)
        large_gaps = time_diff > expected_freq * 2
        
        if large_gaps.any():
            logger.warning(f"Found {large_gaps.sum()} time gaps larger than expected in {timeframe} data")
        
        logger.info(f"Validation completed for {timeframe} data")
        return df
        
    except Exception as e:
        logger.error(f"Error in data validation for {timeframe}: {str(e)}")
        raise

def add_robust_features(df: pd.DataFrame, timeframe: str) -> pd.DataFrame:
    """Add derived features with error handling"""
    try:
        # Calculate returns with handling for division by zero
        df['returns'] = df['close'].pct_change().fillna(0)
        
        # Cap extreme returns
        extreme_returns = np.abs(df['returns']) > 0.5  # 50% moves
        if extreme_returns.any():
            logger.warning(f"Capping {extreme_returns.sum()} extreme returns in {timeframe} data")
            df.loc[extreme_returns, 'returns'] = np.sign(df.loc[extreme_returns, 'returns']) * 0.5
        
        # Calculate log returns safely
        df['log_returns'] = np.log1p(df['returns'])  # log1p is more stable for small values
        
        # Calculate volatility with minimum periods
        df['volatility'] = df['returns'].rolling(window=20, min_periods=5).std().fillna(0)
        
        # Volume metrics with safety checks
        df['volume_sma'] = df['volume'].rolling(window=20, min_periods=5).mean().fillna(df['volume'])
        df['volume_ratio'] = np.where(
            df['volume_sma'] > 0,
            df['volume'] / df['volume_sma'],
            1.0
        )
        
        # Price ranges with safety checks
        df['high_low_range'] = np.where(
            df['close'] > 0,
            (df['high'] - df['low']) / df['close'],
            0
        )
        df['close_open_range'] = np.where(
            df['open'] > 0,
            (df['close'] - df['open']) / df['open'],
            0
        )
        
        # VWAP calculation with cumulative approach
        typical_price = (df['high'] + df['low'] + df['close']) / 3
        df['vwap'] = (typical_price * df['volume']).cumsum() / df['volume'].cumsum()
        df['vwap'] = df['vwap'].fillna(typical_price)  # Handle initial NaN values
        
        # Add trend indicators
        df['sma_20'] = df['close'].rolling(window=20, min_periods=5).mean()
        df['sma_50'] = df['close'].rolling(window=50, min_periods=10).mean()
        df['price_position'] = (df['close'] - df['sma_20']) / df['sma_20']
        
        logger.info(f"Added features to {timeframe} data")
        return df
        
    except Exception as e:
        logger.error(f"Error adding features to {timeframe} data: {str(e)}")
        raise

def align_timeframes(df_30m: pd.DataFrame, df_5m: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Align data timeframes with validation"""
    try:
        # Find common time range
        start_time = max(df_30m.index[0], df_5m.index[0])
        end_time = min(df_30m.index[-1], df_5m.index[-1])
        
        logger.info(f"Aligning data from {start_time} to {end_time}")
        
        # Filter to common range
        df_30m = df_30m[(df_30m.index >= start_time) & (df_30m.index <= end_time)]
        df_5m = df_5m[(df_5m.index >= start_time) & (df_5m.index <= end_time)]
        
        # Validate alignment
        expected_ratio = 6  # 30min / 5min
        actual_ratio = len(df_5m) / len(df_30m)
        
        if abs(actual_ratio - expected_ratio) > 1:
            logger.warning(f"Unexpected data ratio: {actual_ratio:.2f} (expected ~{expected_ratio})")
        
        return df_30m, df_5m
        
    except Exception as e:
        logger.error(f"Error aligning timeframes: {str(e)}")
        raise

# Load the data with error handling
try:
    df_30m, df_5m = load_data()
    logger.info("Data loading successful!")
except Exception as e:
    logger.error(f"Failed to load data: {str(e)}")
    raise

## 2. Advanced NW-RQK Implementation with Multi-Kernel Ensemble

In [ ]:
@njit(fastmath=True, cache=True)
def rational_quadratic_kernel(x1, x2, alpha=0.5, length_scale=50.0):
    """Rational Quadratic Kernel for NW-RQK"""
    # Validate inputs
    if alpha <= 0 or alpha >= 1:
        alpha = 0.5  # Default safe value
    if length_scale <= 0:
        length_scale = 50.0  # Default safe value
    
    diff = x1 - x2
    return (1.0 + (diff * diff) / (2.0 * alpha * length_scale * length_scale)) ** (-alpha)

@njit(fastmath=True, cache=True)
def gaussian_kernel(x1, x2, length_scale=50.0):
    """Gaussian Kernel for ensemble"""
    if length_scale <= 0:
        length_scale = 50.0
    
    diff = x1 - x2
    return np.exp(-0.5 * (diff * diff) / (length_scale * length_scale))

@njit(parallel=True, fastmath=True, cache=True)
def nwrqk_ensemble(prices, window=30, n_kernels=3):
    """Multi-kernel ensemble NW-RQK implementation with validation"""
    n = len(prices)
    nwrqk_values = np.zeros(n)
    
    # Input validation
    if window < 10:
        window = 10
    if n_kernels < 1:
        n_kernels = 3
    
    # Kernel parameters for ensemble
    alphas = np.array([0.3, 0.5, 0.7])
    length_scales = np.array([30.0, 50.0, 70.0])
    
    # Initialize with actual prices for the first window
    for i in range(min(window, n)):
        nwrqk_values[i] = prices[i] if i < len(prices) else prices[-1]
    
    for i in prange(window, n):
        # Window data
        window_prices = prices[i-window:i]
        
        # Check for valid window data
        if np.all(np.isnan(window_prices)) or np.all(window_prices == 0):
            nwrqk_values[i] = nwrqk_values[i-1] if i > 0 else prices[i]
            continue
        
        # Ensemble predictions
        predictions = np.zeros(n_kernels)
        valid_predictions = 0
        
        for k in range(n_kernels):
            # Calculate weights using RQ kernel
            weights = np.zeros(window)
            for j in range(window):
                weights[j] = rational_quadratic_kernel(
                    float(i), float(i-window+j), 
                    alphas[k % len(alphas)], 
                    length_scales[k % len(length_scales)]
                )
            
            # Normalize weights
            weight_sum = np.sum(weights)
            if weight_sum > 1e-10:  # Avoid division by very small numbers
                weights /= weight_sum
                predictions[k] = np.sum(weights * window_prices)
                valid_predictions += 1
            else:
                predictions[k] = np.nan
        
        # Weighted ensemble - only use valid predictions
        if valid_predictions > 0:
            valid_mask = ~np.isnan(predictions)
            nwrqk_values[i] = np.mean(predictions[valid_mask])
        else:
            # Fallback to simple moving average
            nwrqk_values[i] = np.mean(window_prices)
    
    return nwrqk_values

@njit(parallel=True, fastmath=True, cache=True)
def calculate_nwrqk_signals(prices, nwrqk_values, threshold=0.002):
    """Generate NW-RQK trend signals with adaptive thresholds and validation"""
    n = len(prices)
    bull_signals = np.zeros(n, dtype=np.bool_)
    bear_signals = np.zeros(n, dtype=np.bool_)
    signal_strength = np.zeros(n)
    
    # Validate threshold
    if threshold <= 0:
        threshold = 0.002
    
    for i in prange(1, n):
        # Skip if invalid data
        if nwrqk_values[i] <= 0 or prices[i] <= 0 or np.isnan(nwrqk_values[i]) or np.isnan(prices[i]):
            continue
        
        # Price relative to NW-RQK
        deviation = (prices[i] - nwrqk_values[i]) / nwrqk_values[i]
        
        # NW-RQK slope calculation with safety checks
        if i > 5 and nwrqk_values[i-5] > 0:
            slope = (nwrqk_values[i] - nwrqk_values[i-5]) / nwrqk_values[i-5]
            
            # Adaptive threshold based on volatility
            vol_window = 20
            adaptive_threshold = threshold
            
            if i > vol_window:
                returns = np.zeros(vol_window)
                valid_returns = 0
                
                for j in range(vol_window):
                    if i-j-1 >= 0 and prices[i-j-1] > 0 and prices[i-j] > 0:
                        returns[valid_returns] = (prices[i-j] - prices[i-j-1]) / prices[i-j-1]
                        valid_returns += 1
                
                if valid_returns > 5:  # Need minimum returns for volatility
                    volatility = np.std(returns[:valid_returns])
                    # Cap volatility adjustment to prevent extreme thresholds
                    volatility = min(volatility, 0.1)  # Cap at 10% volatility
                    adaptive_threshold = threshold * (1 + volatility * 10)
            
            # Strong trend signals with deviation bounds
            if slope > adaptive_threshold and -0.05 < deviation < 0.05:  # Within 5% of NW-RQK
                bull_signals[i] = True
                signal_strength[i] = min(slope / adaptive_threshold, 2.0)
            elif slope < -adaptive_threshold and -0.05 < deviation < 0.05:
                bear_signals[i] = True
                signal_strength[i] = min(abs(slope) / adaptive_threshold, 2.0)
    
    return bull_signals, bear_signals, signal_strength

# Add validation wrapper for NW-RQK calculation
def calculate_nwrqk_with_validation(prices, config=None):
    """Calculate NW-RQK with comprehensive validation"""
    try:
        if config is None:
            config = StrategyConfig
        
        # Validate input
        if len(prices) < config.NWRQK_WINDOW * 2:
            raise ValueError(f"Insufficient data for NW-RQK: need at least {config.NWRQK_WINDOW * 2} points")
        
        # Check for valid prices
        valid_prices = prices[~np.isnan(prices) & (prices > 0)]
        if len(valid_prices) < len(prices) * 0.9:
            logger.warning(f"More than 10% invalid prices in NW-RQK input")
        
        # Calculate NW-RQK
        logger.info("Calculating NW-RQK ensemble...")
        nwrqk_values = nwrqk_ensemble(
            prices,
            window=config.NWRQK_WINDOW,
            n_kernels=config.NWRQK_N_KERNELS
        )
        
        # Validate output
        if np.all(np.isnan(nwrqk_values)) or np.all(nwrqk_values == 0):
            raise ValueError("NW-RQK calculation failed - all values invalid")
        
        # Calculate signals
        bull_signals, bear_signals, signal_strength = calculate_nwrqk_signals(
            prices, 
            nwrqk_values,
            threshold=config.NWRQK_THRESHOLD
        )
        
        # Log statistics
        logger.info(f"NW-RQK calculation complete - Bull: {bull_signals.sum()}, Bear: {bear_signals.sum()}")
        
        return nwrqk_values, bull_signals, bear_signals, signal_strength
        
    except Exception as e:
        logger.error(f"Error in NW-RQK calculation: {str(e)}")
        # Return safe defaults
        n = len(prices)
        return prices.copy(), np.zeros(n, dtype=bool), np.zeros(n, dtype=bool), np.zeros(n)

## 3. Enhanced MLMI with Volatility-Adaptive KNN

In [ ]:
@njit(fastmath=True, cache=True)
def calculate_rsi(prices, period=14):
    """Ultra-fast RSI calculation with validation"""
    n = len(prices)
    rsi = np.zeros(n)
    
    # Validate period
    if period < 2:
        period = 14
    
    if n < period + 1:
        return rsi
    
    # Calculate price changes
    deltas = np.zeros(n)
    for i in range(1, n):
        if prices[i-1] > 0 and not np.isnan(prices[i]) and not np.isnan(prices[i-1]):
            deltas[i] = prices[i] - prices[i-1]
    
    # Initial averages
    avg_gain = 0.0
    avg_loss = 0.0
    valid_deltas = 0
    
    for i in range(1, min(period + 1, n)):
        if not np.isnan(deltas[i]):
            if deltas[i] > 0:
                avg_gain += deltas[i]
            else:
                avg_loss -= deltas[i]
            valid_deltas += 1
    
    if valid_deltas > 0:
        avg_gain /= valid_deltas
        avg_loss /= valid_deltas
    
    if avg_loss > 0:
        rs = avg_gain / avg_loss
        rsi[period] = 100.0 - (100.0 / (1.0 + rs))
    else:
        rsi[period] = 100.0 if avg_gain > 0 else 50.0
    
    # Calculate RSI for remaining periods
    for i in range(period + 1, n):
        if not np.isnan(deltas[i]):
            if deltas[i] > 0:
                avg_gain = (avg_gain * (period - 1) + deltas[i]) / period
                avg_loss = avg_loss * (period - 1) / period
            else:
                avg_gain = avg_gain * (period - 1) / period
                avg_loss = (avg_loss * (period - 1) - deltas[i]) / period
            
            if avg_loss > 0:
                rs = avg_gain / avg_loss
                rsi[i] = 100.0 - (100.0 / (1.0 + rs))
            else:
                rsi[i] = 100.0 if avg_gain > 0 else 50.0
        else:
            rsi[i] = rsi[i-1] if i > 0 else 50.0
    
    return rsi

@njit(fastmath=True, cache=True)
def euclidean_distance(x1, x2):
    """Calculate Euclidean distance between two vectors with NaN handling"""
    dist = 0.0
    valid_dims = 0
    
    for i in range(len(x1)):
        if not np.isnan(x1[i]) and not np.isnan(x2[i]):
            diff = x1[i] - x2[i]
            dist += diff * diff
            valid_dims += 1
    
    if valid_dims > 0:
        return np.sqrt(dist / valid_dims)  # Normalize by valid dimensions
    else:
        return np.inf  # No valid dimensions

@njit(fastmath=True, cache=True)
def volatility_adaptive_knn(features, labels, query, k_base, volatility, vol_scale=2.0):
    """KNN with volatility-based K adjustment and robustness checks"""
    # Validate inputs
    if k_base < 1:
        k_base = 5
    if volatility < 0:
        volatility = 0
    if vol_scale < 0:
        vol_scale = 2.0
    
    # Adjust K based on volatility
    k = max(3, min(k_base, int(k_base * (1 - min(volatility, 0.5) * vol_scale))))
    
    n_samples = len(labels)
    if n_samples < k:
        # Not enough samples, return neutral
        return 0.5
    
    # Calculate distances
    distances = np.zeros(n_samples)
    valid_samples = 0
    
    for i in range(n_samples):
        dist = euclidean_distance(features[i], query)
        if dist < np.inf:  # Valid distance
            distances[valid_samples] = dist
            valid_samples += 1
    
    if valid_samples < k:
        return 0.5  # Not enough valid samples
    
    # Get k nearest neighbors from valid samples
    indices = np.argsort(distances[:valid_samples])[:k]
    
    # Weighted voting
    bull_score = 0.0
    total_weight = 0.0
    
    for i in range(k):
        idx = indices[i]
        if distances[idx] > 0:
            weight = 1.0 / (1.0 + distances[idx])
        else:
            weight = 1.0
        
        bull_score += labels[idx] * weight
        total_weight += weight
    
    if total_weight > 0:
        return bull_score / total_weight
    else:
        return 0.5

@njit(parallel=True, fastmath=True, cache=True)
def calculate_mlmi_enhanced(prices, window=10, k=5, feature_window=3):
    """Enhanced MLMI with volatility adaptation and robust error handling"""
    n = len(prices)
    mlmi_bull = np.zeros(n, dtype=np.bool_)
    mlmi_bear = np.zeros(n, dtype=np.bool_)
    confidence = np.zeros(n)
    
    # Parameter validation
    if window < 5:
        window = 10
    if k < 3:
        k = 5
    if feature_window < 2:
        feature_window = 3
    
    # Calculate RSI
    rsi = calculate_rsi(prices)
    
    # Calculate volatility
    volatility = np.zeros(n)
    vol_window = 20
    
    for i in range(vol_window, n):
        returns = np.zeros(vol_window)
        valid_returns = 0
        
        for j in range(vol_window):
            if i-j-1 >= 0 and prices[i-j-1] > 0 and prices[i-j] > 0:
                if not np.isnan(prices[i-j]) and not np.isnan(prices[i-j-1]):
                    returns[valid_returns] = (prices[i-j] - prices[i-j-1]) / prices[i-j-1]
                    valid_returns += 1
        
        if valid_returns > 5:  # Need minimum returns
            volatility[i] = np.std(returns[:valid_returns])
        else:
            volatility[i] = 0.02  # Default volatility
    
    # MLMI calculation
    lookback = max(window * 10, 100)
    
    for i in prange(lookback, n):
        # Prepare historical data
        start_idx = max(0, i - lookback)
        historical_size = i - start_idx - feature_window - 1
        
        if historical_size < k:
            continue
        
        # Create feature matrix
        features = np.zeros((historical_size, feature_window))
        labels = np.zeros(historical_size)
        valid_samples = 0
        
        # Fill features and labels
        for j in range(historical_size):
            idx = start_idx + j
            
            # Check if we have valid RSI values for the feature window
            valid_features = True
            for f in range(feature_window):
                if np.isnan(rsi[idx + f]) or rsi[idx + f] <= 0 or rsi[idx + f] >= 100:
                    valid_features = False
                    break
                features[valid_samples, f] = rsi[idx + f]
            
            if not valid_features:
                continue
            
            # Label based on next period return
            if idx + feature_window < n and prices[idx + feature_window] > 0 and prices[idx + feature_window - 1] > 0:
                ret = (prices[idx + feature_window] - prices[idx + feature_window - 1]) / prices[idx + feature_window - 1]
                if not np.isnan(ret) and abs(ret) < 0.5:  # Cap extreme returns
                    labels[valid_samples] = 1.0 if ret > 0 else 0.0
                    valid_samples += 1
        
        if valid_samples < k:
            continue
        
        # Current query
        query = np.zeros(feature_window)
        valid_query = True
        
        for f in range(feature_window):
            if i - feature_window + f >= 0:
                query[f] = rsi[i - feature_window + f]
                if np.isnan(query[f]) or query[f] <= 0 or query[f] >= 100:
                    valid_query = False
                    break
            else:
                valid_query = False
                break
        
        if not valid_query:
            continue
        
        # Adaptive KNN prediction
        bull_prob = volatility_adaptive_knn(
            features[:valid_samples], 
            labels[:valid_samples], 
            query, 
            k, 
            volatility[i], 
            2.0  # vol_scale
        )
        
        confidence[i] = abs(bull_prob - 0.5) * 2  # Convert to confidence score
        
        # Generate signals with confidence threshold
        if bull_prob > 0.65 and confidence[i] > 0.3:
            mlmi_bull[i] = True
        elif bull_prob < 0.35 and confidence[i] > 0.3:
            mlmi_bear[i] = True
    
    return mlmi_bull, mlmi_bear, confidence

# Add validation wrapper for MLMI calculation
def calculate_mlmi_with_validation(prices, config=None):
    """Calculate MLMI with comprehensive validation"""
    try:
        if config is None:
            config = StrategyConfig
        
        # Validate input
        if len(prices) < config.MLMI_LOOKBACK:
            raise ValueError(f"Insufficient data for MLMI: need at least {config.MLMI_LOOKBACK} points")
        
        # Check for valid prices
        valid_prices = prices[~np.isnan(prices) & (prices > 0)]
        if len(valid_prices) < len(prices) * 0.9:
            logger.warning(f"More than 10% invalid prices in MLMI input")
        
        # Calculate MLMI
        logger.info("Calculating MLMI signals...")
        mlmi_bull, mlmi_bear, mlmi_confidence = calculate_mlmi_enhanced(
            prices,
            window=config.MLMI_WINDOW,
            k=config.MLMI_K_NEIGHBORS,
            feature_window=config.MLMI_FEATURE_WINDOW
        )
        
        # Validate output
        total_signals = mlmi_bull.sum() + mlmi_bear.sum()
        if total_signals == 0:
            logger.warning("MLMI generated no signals - check parameters")
        
        # Log statistics
        logger.info(f"MLMI calculation complete - Bull: {mlmi_bull.sum()}, Bear: {mlmi_bear.sum()}")
        
        return mlmi_bull, mlmi_bear, mlmi_confidence
        
    except Exception as e:
        logger.error(f"Error in MLMI calculation: {str(e)}")
        # Return safe defaults
        n = len(prices)
        return np.zeros(n, dtype=bool), np.zeros(n, dtype=bool), np.zeros(n)

## 4. FVG Detection with Volume Confirmation

In [ ]:
@njit(parallel=True, fastmath=True, cache=True)
def detect_fvg_with_volume(high, low, close, volume, min_gap_pct=0.001, volume_factor=1.2):
    """Detect Fair Value Gaps with volume confirmation and robust validation"""
    n = len(high)
    fvg_bull = np.zeros(n, dtype=np.bool_)
    fvg_bear = np.zeros(n, dtype=np.bool_)
    gap_size = np.zeros(n)
    
    # Parameter validation
    if min_gap_pct <= 0:
        min_gap_pct = 0.001
    if volume_factor < 1:
        volume_factor = 1.2
    
    # Calculate average volume with validation
    avg_volume = np.zeros(n)
    vol_window = 20
    
    for i in range(vol_window, n):
        valid_volumes = 0
        vol_sum = 0.0
        
        for j in range(vol_window):
            if i-j >= 0 and volume[i-j] > 0 and not np.isnan(volume[i-j]):
                vol_sum += volume[i-j]
                valid_volumes += 1
        
        if valid_volumes > 5:  # Need minimum valid volumes
            avg_volume[i] = vol_sum / valid_volumes
        else:
            avg_volume[i] = 0  # Will skip this bar
    
    for i in prange(2, n):
        # Skip if no average volume or invalid data
        if avg_volume[i] <= 0:
            continue
        
        # Validate all required data points
        if (np.isnan(high[i]) or np.isnan(low[i]) or np.isnan(close[i]) or
            np.isnan(high[i-2]) or np.isnan(low[i-2]) or np.isnan(close[i-1]) or
            np.isnan(volume[i])):
            continue
        
        # Ensure positive prices
        if close[i-1] <= 0 or high[i] <= 0 or low[i] <= 0 or high[i-2] <= 0 or low[i-2] <= 0:
            continue
        
        # Volume confirmation
        vol_confirmed = volume[i] > avg_volume[i] * volume_factor
        
        # Bullish FVG: gap up
        gap_up = low[i] - high[i-2]
        if gap_up > 0 and vol_confirmed:
            gap_pct = gap_up / close[i-1]
            # Additional validation: gap shouldn't be too large (> 10%)
            if min_gap_pct < gap_pct < 0.1:
                fvg_bull[i] = True
                gap_size[i] = gap_pct
        
        # Bearish FVG: gap down
        gap_down = low[i-2] - high[i]
        if gap_down > 0 and vol_confirmed:
            gap_pct = gap_down / close[i-1]
            # Additional validation: gap shouldn't be too large (> 10%)
            if min_gap_pct < gap_pct < 0.1:
                fvg_bear[i] = True
                gap_size[i] = -gap_pct
    
    return fvg_bull, fvg_bear, gap_size

# Add validation wrapper for FVG calculation
def calculate_fvg_with_validation(df, config=None):
    """Calculate FVG with comprehensive validation"""
    try:
        if config is None:
            config = StrategyConfig
        
        # Validate required columns
        required_columns = ['high', 'low', 'close', 'volume']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns for FVG: {missing_columns}")
        
        # Validate data length
        if len(df) < 50:  # Need enough data for volume average
            raise ValueError(f"Insufficient data for FVG: need at least 50 points")
        
        # Calculate FVG
        logger.info("Calculating FVG signals...")
        fvg_bull, fvg_bear, fvg_size = detect_fvg_with_volume(
            df['high'].values,
            df['low'].values,
            df['close'].values,
            df['volume'].values,
            min_gap_pct=config.FVG_MIN_GAP_PCT,
            volume_factor=config.FVG_VOLUME_FACTOR
        )
        
        # Validate output
        total_gaps = fvg_bull.sum() + fvg_bear.sum()
        if total_gaps == 0:
            logger.warning("No FVG detected - this is normal for some market conditions")
        
        # Check for excessive gaps
        gap_ratio = total_gaps / len(df)
        if gap_ratio > 0.1:  # More than 10% bars have gaps
            logger.warning(f"High number of gaps detected: {gap_ratio:.1%} - consider adjusting parameters")
        
        # Log statistics
        logger.info(f"FVG calculation complete - Bull: {fvg_bull.sum()}, Bear: {fvg_bear.sum()}")
        
        return fvg_bull, fvg_bear, fvg_size
        
    except Exception as e:
        logger.error(f"Error in FVG calculation: {str(e)}")
        # Return safe defaults
        n = len(df)
        return np.zeros(n, dtype=bool), np.zeros(n, dtype=bool), np.zeros(n)

## 5. NW-RQK → MLMI → FVG Synergy Detection

In [ ]:
@njit(parallel=True, fastmath=True, cache=True)
def detect_nwrqk_mlmi_fvg_synergy(nwrqk_bull, nwrqk_bear, nwrqk_strength,
                                  mlmi_bull, mlmi_bear, mlmi_confidence,
                                  fvg_bull, fvg_bear, fvg_size,
                                  window=30):
    """Detect NW-RQK → MLMI → FVG synergy pattern with robust state management"""
    n = len(nwrqk_bull)
    synergy_bull = np.zeros(n, dtype=np.bool_)
    synergy_bear = np.zeros(n, dtype=np.bool_)
    synergy_strength = np.zeros(n)
    
    # Parameter validation
    if window < 10:
        window = 30
    
    # State tracking arrays
    nwrqk_active_bull = np.zeros(n, dtype=np.bool_)
    nwrqk_active_bear = np.zeros(n, dtype=np.bool_)
    mlmi_confirmed_bull = np.zeros(n, dtype=np.bool_)
    mlmi_confirmed_bear = np.zeros(n, dtype=np.bool_)
    
    # State timing for decay
    nwrqk_bull_time = np.full(n, -1, dtype=np.int64)
    nwrqk_bear_time = np.full(n, -1, dtype=np.int64)
    mlmi_bull_time = np.full(n, -1, dtype=np.int64) 
    mlmi_bear_time = np.full(n, -1, dtype=np.int64)
    
    # Track last synergy to prevent duplicate signals
    last_bull_synergy = -window
    last_bear_synergy = -window
    
    for i in prange(1, n):
        # Carry forward states
        if i > 0:
            nwrqk_active_bull[i] = nwrqk_active_bull[i-1]
            nwrqk_active_bear[i] = nwrqk_active_bear[i-1]
            mlmi_confirmed_bull[i] = mlmi_confirmed_bull[i-1]
            mlmi_confirmed_bear[i] = mlmi_confirmed_bear[i-1]
            nwrqk_bull_time[i] = nwrqk_bull_time[i-1]
            nwrqk_bear_time[i] = nwrqk_bear_time[i-1]
            mlmi_bull_time[i] = mlmi_bull_time[i-1]
            mlmi_bear_time[i] = mlmi_bear_time[i-1]
        
        # Step 1: NW-RQK signal activation with strength validation
        if nwrqk_bull[i] and nwrqk_strength[i] > 0.5 and not np.isnan(nwrqk_strength[i]):
            nwrqk_active_bull[i] = True
            nwrqk_active_bear[i] = False
            mlmi_confirmed_bear[i] = False
            nwrqk_bull_time[i] = i
            nwrqk_bear_time[i] = -1
            mlmi_bear_time[i] = -1
        elif nwrqk_bear[i] and nwrqk_strength[i] > 0.5 and not np.isnan(nwrqk_strength[i]):
            nwrqk_active_bear[i] = True
            nwrqk_active_bull[i] = False
            mlmi_confirmed_bull[i] = False
            nwrqk_bear_time[i] = i
            nwrqk_bull_time[i] = -1
            mlmi_bull_time[i] = -1
        
        # Step 2: MLMI confirmation with confidence validation
        if nwrqk_active_bull[i] and mlmi_bull[i] and mlmi_confidence[i] > 0.3 and not np.isnan(mlmi_confidence[i]):
            mlmi_confirmed_bull[i] = True
            mlmi_bull_time[i] = i
        elif nwrqk_active_bear[i] and mlmi_bear[i] and mlmi_confidence[i] > 0.3 and not np.isnan(mlmi_confidence[i]):
            mlmi_confirmed_bear[i] = True
            mlmi_bear_time[i] = i
        
        # Step 3: FVG validation for entry with minimum spacing
        if mlmi_confirmed_bull[i] and fvg_bull[i] and (i - last_bull_synergy) >= 5:
            synergy_bull[i] = True
            last_bull_synergy = i
            
            # Calculate synergy strength with validation
            strength_components = np.zeros(3)
            
            # Find recent NW-RQK strength
            if nwrqk_bull_time[i] >= 0:
                time_since_nwrqk = i - nwrqk_bull_time[i]
                if time_since_nwrqk < window:
                    for j in range(max(0, nwrqk_bull_time[i]), min(i + 1, nwrqk_bull_time[i] + window)):
                        if nwrqk_bull[j] and not np.isnan(nwrqk_strength[j]):
                            strength_components[0] = max(strength_components[0], nwrqk_strength[j])
            
            # MLMI confidence
            if not np.isnan(mlmi_confidence[i]):
                strength_components[1] = mlmi_confidence[i]
            
            # FVG size
            if not np.isnan(fvg_size[i]):
                strength_components[2] = min(abs(fvg_size[i]) * 100, 1.0)
            
            # Calculate weighted strength with validation
            valid_components = 0
            total_strength = 0.0
            for comp in strength_components:
                if comp > 0 and not np.isnan(comp):
                    total_strength += comp
                    valid_components += 1
            
            if valid_components > 0:
                synergy_strength[i] = total_strength / valid_components
            else:
                synergy_strength[i] = 0.5  # Default strength
            
            # Reset states after signal
            nwrqk_active_bull[i] = False
            mlmi_confirmed_bull[i] = False
            nwrqk_bull_time[i] = -1
            mlmi_bull_time[i] = -1
            
        elif mlmi_confirmed_bear[i] and fvg_bear[i] and (i - last_bear_synergy) >= 5:
            synergy_bear[i] = True
            last_bear_synergy = i
            
            # Calculate synergy strength with validation
            strength_components = np.zeros(3)
            
            # Find recent NW-RQK strength
            if nwrqk_bear_time[i] >= 0:
                time_since_nwrqk = i - nwrqk_bear_time[i]
                if time_since_nwrqk < window:
                    for j in range(max(0, nwrqk_bear_time[i]), min(i + 1, nwrqk_bear_time[i] + window)):
                        if nwrqk_bear[j] and not np.isnan(nwrqk_strength[j]):
                            strength_components[0] = max(strength_components[0], nwrqk_strength[j])
            
            # MLMI confidence
            if not np.isnan(mlmi_confidence[i]):
                strength_components[1] = mlmi_confidence[i]
            
            # FVG size
            if not np.isnan(fvg_size[i]):
                strength_components[2] = min(abs(fvg_size[i]) * 100, 1.0)
            
            # Calculate weighted strength with validation
            valid_components = 0
            total_strength = 0.0
            for comp in strength_components:
                if comp > 0 and not np.isnan(comp):
                    total_strength += comp
                    valid_components += 1
            
            if valid_components > 0:
                synergy_strength[i] = total_strength / valid_components
            else:
                synergy_strength[i] = 0.5  # Default strength
            
            # Reset states after signal
            nwrqk_active_bear[i] = False
            mlmi_confirmed_bear[i] = False
            nwrqk_bear_time[i] = -1
            mlmi_bear_time[i] = -1
        
        # State decay - reset if signals are too old
        if nwrqk_bull_time[i] >= 0 and i - nwrqk_bull_time[i] > window:
            nwrqk_active_bull[i] = False
            mlmi_confirmed_bull[i] = False
            nwrqk_bull_time[i] = -1
            mlmi_bull_time[i] = -1
        
        if nwrqk_bear_time[i] >= 0 and i - nwrqk_bear_time[i] > window:
            nwrqk_active_bear[i] = False
            mlmi_confirmed_bear[i] = False
            nwrqk_bear_time[i] = -1
            mlmi_bear_time[i] = -1
    
    return synergy_bull, synergy_bear, synergy_strength

# Add validation wrapper for synergy detection
def detect_synergy_with_validation(nwrqk_data, mlmi_data, fvg_data, config=None):
    """Detect synergies with comprehensive validation"""
    try:
        if config is None:
            config = StrategyConfig
        
        # Validate input lengths match
        n = len(nwrqk_data[0])
        if len(mlmi_data[0]) != n or len(fvg_data[0]) != n:
            raise ValueError("Input data lengths do not match")
        
        # Detect synergies
        logger.info("Detecting NW-RQK → MLMI → FVG synergies...")
        synergy_bull, synergy_bear, synergy_strength = detect_nwrqk_mlmi_fvg_synergy(
            nwrqk_data[0], nwrqk_data[1], nwrqk_data[2],  # bull, bear, strength
            mlmi_data[0], mlmi_data[1], mlmi_data[2],     # bull, bear, confidence
            fvg_data[0], fvg_data[1], fvg_data[2],        # bull, bear, size
            window=config.SYNERGY_WINDOW
        )
        
        # Validate output
        total_synergies = synergy_bull.sum() + synergy_bear.sum()
        if total_synergies == 0:
            logger.warning("No synergies detected - consider adjusting parameters")
        
        # Check synergy rate
        synergy_rate = total_synergies / n
        if synergy_rate > 0.1:  # More than 10% bars have synergies
            logger.warning(f"High synergy rate: {synergy_rate:.1%} - may indicate overfitting")
        
        # Log statistics
        logger.info(f"Synergy detection complete - Bull: {synergy_bull.sum()}, Bear: {synergy_bear.sum()}")
        avg_strength = synergy_strength[synergy_strength > 0].mean() if (synergy_strength > 0).any() else 0
        logger.info(f"Average synergy strength: {avg_strength:.3f}")
        
        return synergy_bull, synergy_bear, synergy_strength
        
    except Exception as e:
        logger.error(f"Error in synergy detection: {str(e)}")
        # Return safe defaults
        return np.zeros(n, dtype=bool), np.zeros(n, dtype=bool), np.zeros(n)

## 6. Complete Strategy Implementation

In [ ]:
def run_nwrqk_mlmi_fvg_strategy(df_30m, df_5m):
    """Execute the complete NW-RQK → MLMI → FVG strategy with robust error handling"""
    logger.info("\n" + "="*60)
    logger.info("NW-RQK → MLMI → FVG SYNERGY STRATEGY")
    logger.info("="*60)
    
    start_time = time.time()
    
    try:
        # Performance tracking
        performance_metrics = {
            'start_time': datetime.now(),
            'errors': [],
            'warnings': []
        }
        
        # 1. Calculate NW-RQK signals with validation
        logger.info("\n1. Calculating NW-RQK signals...")
        nwrqk_calc_start = time.time()
        
        prices = df_30m['close'].values
        nwrqk_values, nwrqk_bull, nwrqk_bear, nwrqk_strength = calculate_nwrqk_with_validation(
            prices, StrategyConfig
        )
        
        performance_metrics['nwrqk_time'] = time.time() - nwrqk_calc_start
        logger.info(f"   - NW-RQK calculation time: {performance_metrics['nwrqk_time']:.2f}s")
        logger.info(f"   - Bull signals: {nwrqk_bull.sum()}")
        logger.info(f"   - Bear signals: {nwrqk_bear.sum()}")
        
        # 2. Calculate MLMI signals with validation
        logger.info("\n2. Calculating MLMI signals...")
        mlmi_calc_start = time.time()
        
        mlmi_bull, mlmi_bear, mlmi_confidence = calculate_mlmi_with_validation(
            prices, StrategyConfig
        )
        
        performance_metrics['mlmi_time'] = time.time() - mlmi_calc_start
        logger.info(f"   - MLMI calculation time: {performance_metrics['mlmi_time']:.2f}s")
        logger.info(f"   - Bull signals: {mlmi_bull.sum()}")
        logger.info(f"   - Bear signals: {mlmi_bear.sum()}")
        
        # 3. Calculate FVG on 5-minute data with validation
        logger.info("\n3. Calculating FVG signals on 5m data...")
        fvg_calc_start = time.time()
        
        # Check if FVG columns exist to avoid recalculation
        if 'fvg_bull' not in df_5m.columns:
            fvg_bull_5m, fvg_bear_5m, fvg_size_5m = calculate_fvg_with_validation(
                df_5m, StrategyConfig
            )
            df_5m['fvg_bull'] = fvg_bull_5m
            df_5m['fvg_bear'] = fvg_bear_5m
            df_5m['fvg_size'] = fvg_size_5m
        
        performance_metrics['fvg_time'] = time.time() - fvg_calc_start
        logger.info(f"   - FVG calculation time: {performance_metrics['fvg_time']:.2f}s")
        logger.info(f"   - Bull FVGs: {df_5m['fvg_bull'].sum()}")
        logger.info(f"   - Bear FVGs: {df_5m['fvg_bear'].sum()}")
        
        # 4. Map 5m FVG to 30m timeframe with validation
        logger.info("\n4. Mapping FVG signals to 30m timeframe...")
        
        try:
            # Resample FVG signals
            fvg_resampled = df_5m[['fvg_bull', 'fvg_bear', 'fvg_size']].resample('30min').agg({
                'fvg_bull': 'max',
                'fvg_bear': 'max',
                'fvg_size': 'mean'
            })
            
            # Align with 30m data
            fvg_aligned = fvg_resampled.reindex(df_30m.index, method='ffill')
            fvg_aligned = fvg_aligned.fillna(False)
            
            # Validate alignment
            if len(fvg_aligned) != len(df_30m):
                raise ValueError("FVG alignment failed - length mismatch")
                
        except Exception as e:
            logger.error(f"Error in FVG resampling: {str(e)}")
            # Create empty FVG signals as fallback
            fvg_aligned = pd.DataFrame(index=df_30m.index)
            fvg_aligned['fvg_bull'] = False
            fvg_aligned['fvg_bear'] = False
            fvg_aligned['fvg_size'] = 0
            performance_metrics['errors'].append(f"FVG resampling error: {str(e)}")
        
        # 5. Detect synergies with validation
        logger.info("\n5. Detecting NW-RQK → MLMI → FVG synergies...")
        synergy_calc_start = time.time()
        
        synergy_bull, synergy_bear, synergy_strength = detect_synergy_with_validation(
            (nwrqk_bull, nwrqk_bear, nwrqk_strength),
            (mlmi_bull, mlmi_bear, mlmi_confidence),
            (fvg_aligned['fvg_bull'].values.astype(np.bool_),
             fvg_aligned['fvg_bear'].values.astype(np.bool_),
             fvg_aligned['fvg_size'].fillna(0).values),
            StrategyConfig
        )
        
        performance_metrics['synergy_time'] = time.time() - synergy_calc_start
        logger.info(f"   - Synergy detection time: {performance_metrics['synergy_time']:.2f}s")
        logger.info(f"   - Bull synergies: {synergy_bull.sum()}")
        logger.info(f"   - Bear synergies: {synergy_bear.sum()}")
        logger.info(f"   - Total signals: {synergy_bull.sum() + synergy_bear.sum()}")
        
        # 6. Create signals DataFrame with risk management
        signals = pd.DataFrame(index=df_30m.index)
        signals['synergy_bull'] = synergy_bull
        signals['synergy_bear'] = synergy_bear
        signals['synergy_strength'] = synergy_strength
        signals['price'] = df_30m['close']
        
        # Add volatility for risk management
        signals['volatility'] = df_30m['volatility']
        
        # Generate position signals with risk filters
        signals['signal'] = 0
        
        # Apply risk filters
        max_volatility = signals['volatility'].quantile(0.95)
        min_strength = 0.3
        
        # Bull signals with risk filters
        valid_bull = (signals['synergy_bull'] & 
                     (signals['volatility'] < max_volatility) & 
                     (signals['synergy_strength'] > min_strength))
        signals.loc[valid_bull, 'signal'] = 1
        
        # Bear signals with risk filters  
        valid_bear = (signals['synergy_bear'] & 
                     (signals['volatility'] < max_volatility) & 
                     (signals['synergy_strength'] > min_strength))
        signals.loc[valid_bear, 'signal'] = -1
        
        # Add signal quality metrics
        signals['signal_quality'] = signals['synergy_strength'] * (1 - signals['volatility'] / max_volatility)
        
        # Performance summary
        performance_metrics['total_time'] = time.time() - start_time
        performance_metrics['signals_generated'] = (signals['signal'] != 0).sum()
        performance_metrics['signals_filtered'] = (synergy_bull.sum() + synergy_bear.sum()) - performance_metrics['signals_generated']
        
        logger.info(f"\nTotal execution time: {performance_metrics['total_time']:.2f} seconds")
        logger.info(f"Signals after risk filtering: {performance_metrics['signals_generated']}")
        logger.info(f"Signals filtered by risk management: {performance_metrics['signals_filtered']}")
        
        # Store performance metrics in signals
        signals.attrs['performance_metrics'] = performance_metrics
        
        return signals
        
    except Exception as e:
        logger.error(f"Critical error in strategy execution: {str(e)}")
        logger.error(f"Traceback: {traceback.format_exc()}")
        
        # Return empty signals on error
        signals = pd.DataFrame(index=df_30m.index)
        signals['signal'] = 0
        signals['price'] = df_30m['close']
        signals.attrs['error'] = str(e)
        return signals

# Import traceback for error handling
import traceback

# Run the strategy with error handling
try:
    signals = run_nwrqk_mlmi_fvg_strategy(df_30m, df_5m)
    logger.info("Strategy execution completed successfully")
except Exception as e:
    logger.error(f"Failed to run strategy: {str(e)}")
    raise

## 7. VectorBT Backtesting with Advanced Features

In [ ]:
def run_vectorbt_backtest(signals, initial_capital=100000, position_size=0.1, 
                         sl_pct=0.02, tp_pct=0.03, fees=0.001):
    """Run VectorBT backtest with dynamic position sizing and risk management"""
    logger.info("\n" + "="*60)
    logger.info("VECTORBT BACKTEST WITH RISK MANAGEMENT")
    logger.info("="*60)
    
    backtest_start = time.time()
    
    try:
        # Validate signals
        if signals is None or len(signals) == 0:
            raise ValueError("Invalid signals data")
        
        if 'signal' not in signals.columns or 'price' not in signals.columns:
            raise ValueError("Signals must contain 'signal' and 'price' columns")
        
        # Prepare data
        price = signals['price']
        entries = signals['signal'] == 1
        exits = signals['signal'] == -1
        
        # Check if we have any signals
        if not entries.any() and not exits.any():
            logger.warning("No trading signals generated - check strategy parameters")
            return None, None
        
        # Dynamic position sizing based on signal strength and volatility
        if 'synergy_strength' in signals.columns and 'volatility' in signals.columns:
            # Scale position size by signal strength (0.5 to 1.5x base size)
            strength_factor = 0.5 + 0.5 * np.minimum(signals['synergy_strength'], 1.0)
            
            # Reduce position size in high volatility (0.5 to 1.0x)
            vol_percentile = signals['volatility'].rolling(252).apply(
                lambda x: stats.rankdata(x)[-1] / len(x) if len(x) > 0 else 0.5
            ).fillna(0.5)
            vol_factor = 1.0 - 0.5 * vol_percentile
            
            # Combined position sizing
            position_sizes = position_size * strength_factor * vol_factor
            position_sizes = position_sizes.fillna(position_size)
            
            # Apply position size limits
            position_sizes = np.clip(position_sizes, 
                                   position_size * 0.5,  # Min 50% of base
                                   position_size * 1.5)  # Max 150% of base
        else:
            position_sizes = position_size
        
        # Risk management parameters
        sl_pct = StrategyConfig.STOP_LOSS_PCT
        tp_pct = StrategyConfig.TAKE_PROFIT_PCT
        
        # Run backtest with VectorBT
        portfolio = vbt.Portfolio.from_signals(
            price,
            entries=entries,
            exits=exits,
            size=position_sizes,
            size_type='percent',
            init_cash=initial_capital,
            fees=fees,
            slippage=StrategyConfig.SLIPPAGE,
            freq='30min',
            sl_stop=sl_pct,
            tp_stop=tp_pct,
            stop_exit_price='close',  # Use close price for stops
            upon_stop_exit='close_position',  # Close full position on stop
            raise_reject=False,  # Don't raise on rejected orders
            log=True  # Enable logging for debugging
        )
        
        # Calculate metrics with error handling
        try:
            stats = portfolio.stats()
            
            # Additional risk metrics
            returns = portfolio.returns()
            
            # Maximum consecutive losses
            trade_returns = portfolio.trades.records_readable['Return [%]'].values
            losing_trades = trade_returns < 0
            max_consecutive_losses = 0
            current_losses = 0
            for loss in losing_trades:
                if loss:
                    current_losses += 1
                    max_consecutive_losses = max(max_consecutive_losses, current_losses)
                else:
                    current_losses = 0
            
            # Risk-adjusted metrics
            downside_returns = returns[returns < 0]
            if len(downside_returns) > 0:
                downside_std = downside_returns.std()
                sortino_ratio = (returns.mean() / downside_std) * np.sqrt(252 * 48) if downside_std > 0 else 0
            else:
                sortino_ratio = np.inf
            
            # Add custom metrics to stats
            stats['Max Consecutive Losses'] = max_consecutive_losses
            stats['Sortino Ratio (Custom)'] = sortino_ratio
            stats['Average Position Size'] = position_sizes.mean() if isinstance(position_sizes, pd.Series) else position_size
            
        except Exception as e:
            logger.error(f"Error calculating portfolio statistics: {str(e)}")
            stats = {'Error': str(e)}
        
        logger.info(f"\nBacktest execution time: {time.time() - backtest_start:.2f} seconds")
        
        # Log key performance metrics
        if 'Total Return [%]' in stats:
            logger.info("\nKey Performance Metrics:")
            logger.info(f"Total Return: {stats.get('Total Return [%]', 'N/A'):.2f}%")
            logger.info(f"Sharpe Ratio: {stats.get('Sharpe Ratio', 'N/A'):.2f}")
            logger.info(f"Max Drawdown: {stats.get('Max Drawdown [%]', 'N/A'):.2f}%")
            logger.info(f"Win Rate: {stats.get('Win Rate [%]', 'N/A'):.2f}%")
            logger.info(f"Total Trades: {stats.get('Total Trades', 'N/A')}")
            
            # Calculate annual metrics
            if price.index[-1] and price.index[0]:
                n_years = (price.index[-1] - price.index[0]).days / 365.25
                if n_years > 0 and stats.get('Total Return [%]'):
                    annual_return = (1 + stats['Total Return [%]'] / 100) ** (1 / n_years) - 1
                    trades_per_year = stats.get('Total Trades', 0) / n_years
                    
                    logger.info(f"\nAnnualized Return: {annual_return * 100:.2f}%")
                    logger.info(f"Trades per Year: {trades_per_year:.0f}")
            
            # Risk warnings
            if stats.get('Max Drawdown [%]', 0) > StrategyConfig.MAX_DRAWDOWN_LIMIT * 100:
                logger.warning(f"⚠️  Maximum drawdown exceeds limit: {stats['Max Drawdown [%]']:.2f}% > {StrategyConfig.MAX_DRAWDOWN_LIMIT * 100:.0f}%")
            
            if stats.get('Win Rate [%]', 0) < 40:
                logger.warning(f"⚠️  Low win rate: {stats['Win Rate [%]']:.2f}%")
        
        return portfolio, stats
        
    except Exception as e:
        logger.error(f"Critical error in backtesting: {str(e)}")
        logger.error(f"Traceback: {traceback.format_exc()}")
        return None, {'Error': str(e)}

# Run backtest with error handling
try:
    portfolio, stats = run_vectorbt_backtest(signals)
    if portfolio is not None:
        logger.info("Backtest completed successfully")
    else:
        logger.error("Backtest failed - check logs for details")
except Exception as e:
    logger.error(f"Failed to run backtest: {str(e)}")
    portfolio, stats = None, {'Error': str(e)}

## 8. Advanced Visualizations

In [ ]:
def create_performance_dashboard(signals, portfolio):
    """Create comprehensive performance dashboard"""
    # Create subplots
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            'Portfolio Value', 'Monthly Returns',
            'Cumulative Returns', 'Drawdown',
            'Trade Distribution', 'Signal Strength vs Returns',
            'Rolling Sharpe Ratio', 'Win Rate by Month'
        ),
        row_heights=[0.25, 0.25, 0.25, 0.25],
        specs=[
            [{"secondary_y": False}, {"type": "bar"}],
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"type": "histogram"}, {"type": "scatter"}],
            [{"secondary_y": False}, {"type": "bar"}]
        ]
    )
    
    # 1. Portfolio Value
    fig.add_trace(
        go.Scatter(
            x=portfolio.value().index,
            y=portfolio.value().values,
            name='Portfolio Value',
            line=dict(color='cyan', width=2)
        ),
        row=1, col=1
    )
    
    # 2. Monthly Returns
    monthly_returns = portfolio.returns().resample('M').apply(lambda x: (1 + x).prod() - 1)
    colors = ['green' if r > 0 else 'red' for r in monthly_returns]
    fig.add_trace(
        go.Bar(
            x=monthly_returns.index,
            y=monthly_returns.values * 100,
            name='Monthly Returns',
            marker_color=colors
        ),
        row=1, col=2
    )
    
    # 3. Cumulative Returns
    cum_returns = (1 + portfolio.returns()).cumprod() - 1
    fig.add_trace(
        go.Scatter(
            x=cum_returns.index,
            y=cum_returns.values * 100,
            name='Cumulative Returns',
            fill='tozeroy',
            line=dict(color='lightblue')
        ),
        row=2, col=1
    )
    
    # 4. Drawdown
    drawdown = portfolio.drawdown() * 100
    fig.add_trace(
        go.Scatter(
            x=drawdown.index,
            y=-drawdown.values,
            name='Drawdown',
            fill='tozeroy',
            line=dict(color='red')
        ),
        row=2, col=2
    )
    
    # 5. Trade Distribution
    trade_returns = portfolio.trades.records_readable['Return [%]'].values
    fig.add_trace(
        go.Histogram(
            x=trade_returns,
            nbinsx=50,
            name='Trade Returns',
            marker_color='purple'
        ),
        row=3, col=1
    )
    
    # 6. Signal Strength vs Returns
    trade_records = portfolio.trades.records_readable
    entry_times = pd.to_datetime(trade_records['Entry Timestamp'])
    signal_strengths = []
    for entry_time in entry_times:
        idx = signals.index.get_indexer([entry_time], method='nearest')[0]
        if idx < len(signals):
            signal_strengths.append(signals.iloc[idx]['synergy_strength'])
        else:
            signal_strengths.append(0)
    
    fig.add_trace(
        go.Scatter(
            x=signal_strengths,
            y=trade_returns,
            mode='markers',
            name='Strength vs Return',
            marker=dict(
                size=5,
                color=trade_returns,
                colorscale='RdYlGn',
                showscale=True
            )
        ),
        row=3, col=2
    )
    
    # 7. Rolling Sharpe Ratio
    rolling_sharpe = portfolio.sharpe_ratio(rolling=252)
    fig.add_trace(
        go.Scatter(
            x=rolling_sharpe.index,
            y=rolling_sharpe.values,
            name='Rolling Sharpe',
            line=dict(color='orange')
        ),
        row=4, col=1
    )
    
    # 8. Win Rate by Month
    trades_df = trade_records.copy()
    trades_df['Month'] = pd.to_datetime(trades_df['Entry Timestamp']).dt.to_period('M')
    monthly_stats = trades_df.groupby('Month').agg({
        'Return [%]': ['count', lambda x: (x > 0).sum() / len(x) * 100]
    })
    monthly_stats.columns = ['Count', 'Win Rate']
    
    fig.add_trace(
        go.Bar(
            x=monthly_stats.index.astype(str),
            y=monthly_stats['Win Rate'],
            name='Win Rate %',
            marker_color='lightgreen'
        ),
        row=4, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text="NW-RQK → MLMI → FVG Synergy Performance Dashboard",
        showlegend=False,
        height=1600,
        template='plotly_dark'
    )
    
    # Update axes
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=2)
    fig.update_xaxes(title_text="Return %", row=3, col=1)
    fig.update_xaxes(title_text="Signal Strength", row=3, col=2)
    fig.update_xaxes(title_text="Date", row=4, col=1)
    fig.update_xaxes(title_text="Month", row=4, col=2)
    
    fig.update_yaxes(title_text="Value ($)", row=1, col=1)
    fig.update_yaxes(title_text="Return %", row=1, col=2)
    fig.update_yaxes(title_text="Return %", row=2, col=1)
    fig.update_yaxes(title_text="Drawdown %", row=2, col=2)
    fig.update_yaxes(title_text="Frequency", row=3, col=1)
    fig.update_yaxes(title_text="Return %", row=3, col=2)
    fig.update_yaxes(title_text="Sharpe Ratio", row=4, col=1)
    fig.update_yaxes(title_text="Win Rate %", row=4, col=2)
    
    fig.show()
    
    return fig

# Create dashboard
dashboard = create_performance_dashboard(signals, portfolio)

## 9. Monte Carlo Validation

In [ ]:
@njit(parallel=True, fastmath=True)
def monte_carlo_simulation(returns, n_simulations=1000, n_periods=252):
    """Run Monte Carlo simulation for confidence intervals with validation"""
    n_returns = len(returns)
    
    # Validate inputs
    if n_returns < 10:
        raise ValueError("Insufficient returns for Monte Carlo simulation")
    
    if n_simulations < 100:
        n_simulations = 100
    
    if n_periods < 10:
        n_periods = 252
    
    final_values = np.zeros(n_simulations)
    
    # Filter out extreme returns to avoid unrealistic simulations
    valid_returns = returns[~np.isnan(returns)]
    valid_returns = valid_returns[np.abs(valid_returns) < 0.5]  # Cap at 50% moves
    
    if len(valid_returns) < 10:
        raise ValueError("Insufficient valid returns after filtering")
    
    for sim in prange(n_simulations):
        # Bootstrap sample returns
        sim_returns = np.zeros(n_periods)
        for i in range(n_periods):
            idx = np.random.randint(0, len(valid_returns))
            sim_returns[i] = valid_returns[idx]
        
        # Calculate final value with compound returns
        cumulative_return = 1.0
        for ret in sim_returns:
            cumulative_return *= (1 + ret)
        
        final_values[sim] = cumulative_return
    
    return final_values

def run_monte_carlo_analysis(portfolio):
    """Run Monte Carlo analysis for strategy validation with comprehensive checks"""
    logger.info("\n" + "="*60)
    logger.info("MONTE CARLO VALIDATION")
    logger.info("="*60)
    
    mc_start = time.time()
    
    try:
        # Validate portfolio
        if portfolio is None:
            logger.error("No portfolio provided for Monte Carlo analysis")
            return None, None
        
        # Get trade returns with validation
        try:
            trades_df = portfolio.trades.records_readable
            if len(trades_df) == 0:
                logger.error("No trades found in portfolio")
                return None, None
            
            trade_returns = trades_df['Return [%]'].values / 100
            
            # Filter out invalid returns
            valid_mask = ~np.isnan(trade_returns) & (np.abs(trade_returns) < 5.0)  # Cap at 500% moves
            trade_returns = trade_returns[valid_mask]
            
            if len(trade_returns) < 10:
                logger.error(f"Insufficient valid trades for Monte Carlo: {len(trade_returns)}")
                return None, None
            
        except Exception as e:
            logger.error(f"Error extracting trade returns: {str(e)}")
            return None, None
        
        # Run simulation with error handling
        try:
            final_values = monte_carlo_simulation(trade_returns, n_simulations=10000)
        except Exception as e:
            logger.error(f"Error in Monte Carlo simulation: {str(e)}")
            # Try with fewer simulations
            try:
                logger.warning("Retrying with 1000 simulations...")
                final_values = monte_carlo_simulation(trade_returns, n_simulations=1000)
            except:
                return None, None
        
        # Calculate statistics
        mc_returns = (final_values - 1) * 100
        percentiles = np.percentile(mc_returns, [5, 25, 50, 75, 95])
        
        logger.info(f"\nMonte Carlo simulation completed in {time.time() - mc_start:.2f} seconds")
        logger.info(f"Based on {len(trade_returns)} historical trades")
        logger.info("\nConfidence Intervals for Annual Returns:")
        logger.info(f"5th percentile:  {percentiles[0]:.2f}%")
        logger.info(f"25th percentile: {percentiles[1]:.2f}%")
        logger.info(f"Median:          {percentiles[2]:.2f}%")
        logger.info(f"75th percentile: {percentiles[3]:.2f}%")
        logger.info(f"95th percentile: {percentiles[4]:.2f}%")
        
        # Risk metrics
        prob_profit = (mc_returns > 0).mean() * 100
        prob_loss_10pct = (mc_returns < -10).mean() * 100
        prob_gain_20pct = (mc_returns > 20).mean() * 100
        expected_return = mc_returns.mean()
        return_std = mc_returns.std()
        
        logger.info(f"\nRisk Metrics:")
        logger.info(f"Probability of Profit: {prob_profit:.1f}%")
        logger.info(f"Probability of >20% Gain: {prob_gain_20pct:.1f}%")
        logger.info(f"Probability of >10% Loss: {prob_loss_10pct:.1f}%")
        logger.info(f"Expected Return: {expected_return:.2f}%")
        logger.info(f"Return Std Dev: {return_std:.2f}%")
        
        # Create visualization with error handling
        try:
            fig = go.Figure()
            
            # Add histogram
            fig.add_trace(go.Histogram(
                x=mc_returns,
                nbinsx=100,
                name='Simulated Returns',
                marker_color='lightblue',
                opacity=0.7,
                showlegend=False
            ))
            
            # Add percentile lines
            colors = ['red', 'orange', 'green', 'orange', 'red']
            for i, (p, label) in enumerate(zip(percentiles, ['5%', '25%', '50%', '75%', '95%'])):
                fig.add_vline(
                    x=p, 
                    line_dash="dash", 
                    line_color=colors[i],
                    annotation_text=f"{label}: {p:.1f}%",
                    annotation_position="top"
                )
            
            # Add zero line
            fig.add_vline(x=0, line_dash="solid", line_color="black", line_width=2)
            
            fig.update_layout(
                title=f"Monte Carlo Simulation Results ({len(final_values):,} simulations)",
                xaxis_title="Annual Return %",
                yaxis_title="Frequency",
                template='plotly_dark',
                height=600,
                showlegend=False
            )
            
            fig.show()
            
        except Exception as e:
            logger.error(f"Error creating Monte Carlo visualization: {str(e)}")
        
        return mc_returns, percentiles
        
    except Exception as e:
        logger.error(f"Critical error in Monte Carlo analysis: {str(e)}")
        logger.error(f"Traceback: {traceback.format_exc()}")
        return None, None

# Run Monte Carlo validation with error handling
if portfolio is not None:
    try:
        mc_returns, percentiles = run_monte_carlo_analysis(portfolio)
        if mc_returns is not None:
            logger.info("Monte Carlo analysis completed successfully")
    except Exception as e:
        logger.error(f"Failed to run Monte Carlo analysis: {str(e)}")
        mc_returns, percentiles = None, None
else:
    logger.warning("Skipping Monte Carlo analysis - no valid portfolio")

## 10. Summary Statistics and Trade Analysis

In [ ]:
def generate_comprehensive_report(signals, portfolio, stats):
    """Generate comprehensive performance report"""
    print("\n" + "="*60)
    print("COMPREHENSIVE PERFORMANCE REPORT")
    print("="*60)
    
    # Time period analysis
    start_date = signals.index[0]
    end_date = signals.index[-1]
    n_years = (end_date - start_date).days / 365.25
    
    print(f"\nBacktest Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"Duration: {n_years:.1f} years")
    
    # Trade analysis
    trades = portfolio.trades.records_readable
    total_trades = len(trades)
    winning_trades = len(trades[trades['Return [%]'] > 0])
    losing_trades = len(trades[trades['Return [%]'] < 0])
    
    print(f"\nTrade Statistics:")
    print(f"Total Trades: {total_trades}")
    print(f"Trades per Year: {total_trades / n_years:.0f}")
    print(f"Winning Trades: {winning_trades}")
    print(f"Losing Trades: {losing_trades}")
    print(f"Win Rate: {(winning_trades / total_trades * 100) if total_trades > 0 else 0:.2f}%")
    
    # Return analysis
    avg_win = trades[trades['Return [%]'] > 0]['Return [%]'].mean() if winning_trades > 0 else 0
    avg_loss = trades[trades['Return [%]'] < 0]['Return [%]'].mean() if losing_trades > 0 else 0
    profit_factor = abs(avg_win * winning_trades / (avg_loss * losing_trades)) if losing_trades > 0 else np.inf
    
    print(f"\nReturn Metrics:")
    print(f"Average Win: {avg_win:.2f}%")
    print(f"Average Loss: {avg_loss:.2f}%")
    print(f"Profit Factor: {profit_factor:.2f}")
    print(f"Expectancy: {trades['Return [%]'].mean():.2f}%")
    
    # Risk metrics
    print(f"\nRisk Metrics:")
    print(f"Maximum Drawdown: {stats['Max Drawdown [%]']:.2f}%")
    print(f"Average Drawdown: {portfolio.drawdown().mean() * 100:.2f}%")
    print(f"Calmar Ratio: {stats['Calmar Ratio']:.2f}")
    print(f"Sortino Ratio: {stats['Sortino Ratio']:.2f}")
    
    # Signal quality analysis
    bull_signals = signals[signals['synergy_bull']]
    bear_signals = signals[signals['synergy_bear']]
    
    print(f"\nSignal Analysis:")
    print(f"Total Bull Signals: {len(bull_signals)}")
    print(f"Total Bear Signals: {len(bear_signals)}")
    print(f"Average Signal Strength: {signals['synergy_strength'][signals['synergy_strength'] > 0].mean():.3f}")
    
    # Monthly performance
    monthly_returns = portfolio.returns().resample('M').apply(lambda x: (1 + x).prod() - 1)
    positive_months = (monthly_returns > 0).sum()
    total_months = len(monthly_returns)
    
    print(f"\nMonthly Performance:")
    print(f"Positive Months: {positive_months}/{total_months} ({positive_months/total_months*100:.1f}%)")
    print(f"Best Month: {monthly_returns.max() * 100:.2f}%")
    print(f"Worst Month: {monthly_returns.min() * 100:.2f}%")
    print(f"Average Monthly Return: {monthly_returns.mean() * 100:.2f}%")
    
    return trades

# Generate report
trades_df = generate_comprehensive_report(signals, portfolio, stats)

## 11. Export Results

In [ ]:
# Save results with comprehensive error handling
logger.info("\n" + "="*60)
logger.info("SAVING RESULTS")
logger.info("="*60)

# Create results directory if it doesn't exist
results_dir = '/home/QuantNova/AlgoSpace-8/results'
try:
    os.makedirs(results_dir, exist_ok=True)
    logger.info(f"Results directory verified: {results_dir}")
except Exception as e:
    logger.error(f"Failed to create results directory: {str(e)}")
    results_dir = '.'  # Fallback to current directory

# Prepare timestamp for unique filenames
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save signals with error handling
try:
    signals_file = os.path.join(results_dir, f'synergy_3_nwrqk_mlmi_fvg_signals_{timestamp}.csv')
    signals.to_csv(signals_file)
    logger.info(f"✓ Signals saved to {signals_file}")
except Exception as e:
    logger.error(f"Failed to save signals: {str(e)}")

# Save trade records with validation
if portfolio is not None:
    try:
        trades_df = portfolio.trades.records_readable
        if len(trades_df) > 0:
            trades_file = os.path.join(results_dir, f'synergy_3_nwrqk_mlmi_fvg_trades_{timestamp}.csv')
            trades_df.to_csv(trades_file)
            logger.info(f"✓ Trade records saved to {trades_file}")
        else:
            logger.warning("No trades to save")
    except Exception as e:
        logger.error(f"Failed to save trade records: {str(e)}")
else:
    logger.warning("No portfolio available - skipping trade records")

# Save performance metrics with comprehensive details
try:
    metrics_file = os.path.join(results_dir, f'synergy_3_nwrqk_mlmi_fvg_metrics_{timestamp}.txt')
    with open(metrics_file, 'w') as f:
        f.write("="*60 + "\n")
        f.write("NW-RQK → MLMI → FVG SYNERGY STRATEGY PERFORMANCE REPORT\n")
        f.write("="*60 + "\n\n")
        
        # Strategy configuration
        f.write("STRATEGY CONFIGURATION:\n")
        f.write("-"*30 + "\n")
        f.write(f"Run Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Data Period: {df_30m.index[0]} to {df_30m.index[-1]}\n")
        f.write(f"Initial Capital: ${StrategyConfig.INITIAL_CAPITAL:,.2f}\n")
        f.write(f"Position Size: {StrategyConfig.POSITION_SIZE_BASE * 100:.1f}%\n")
        f.write(f"Stop Loss: {StrategyConfig.STOP_LOSS_PCT * 100:.1f}%\n")
        f.write(f"Take Profit: {StrategyConfig.TAKE_PROFIT_PCT * 100:.1f}%\n")
        f.write(f"Trading Fees: {StrategyConfig.TRADING_FEES * 100:.2f}%\n")
        f.write(f"Slippage: {StrategyConfig.SLIPPAGE * 100:.2f}%\n\n")
        
        # Performance metrics
        if stats is not None and 'Error' not in stats:
            f.write("PERFORMANCE METRICS:\n")
            f.write("-"*30 + "\n")
            for key, value in stats.items():
                if isinstance(value, (int, float)):
                    if 'Return' in key or 'Ratio' in key or '%' in key:
                        f.write(f"{key}: {value:.2f}\n")
                    else:
                        f.write(f"{key}: {value:.4f}\n")
                else:
                    f.write(f"{key}: {value}\n")
        else:
            f.write("Performance metrics unavailable due to errors\n")
        
        # Signal statistics
        f.write("\nSIGNAL STATISTICS:\n")
        f.write("-"*30 + "\n")
        f.write(f"Total Signals Generated: {(signals['signal'] != 0).sum()}\n")
        f.write(f"Bull Signals: {(signals['signal'] == 1).sum()}\n")
        f.write(f"Bear Signals: {(signals['signal'] == -1).sum()}\n")
        
        if 'synergy_strength' in signals.columns:
            avg_strength = signals.loc[signals['signal'] != 0, 'synergy_strength'].mean()
            f.write(f"Average Signal Strength: {avg_strength:.3f}\n")
        
        # Risk analysis
        if hasattr(signals, 'attrs') and 'performance_metrics' in signals.attrs:
            perf = signals.attrs['performance_metrics']
            f.write("\nPERFORMANCE ANALYSIS:\n")
            f.write("-"*30 + "\n")
            f.write(f"Total Execution Time: {perf.get('total_time', 0):.2f} seconds\n")
            f.write(f"Signals Filtered by Risk: {perf.get('signals_filtered', 0)}\n")
            
            if perf.get('errors'):
                f.write("\nERRORS ENCOUNTERED:\n")
                for error in perf['errors']:
                    f.write(f"- {error}\n")
        
        # Monte Carlo results
        if percentiles is not None:
            f.write("\nMONTE CARLO ANALYSIS:\n")
            f.write("-"*30 + "\n")
            f.write(f"5th Percentile Return: {percentiles[0]:.2f}%\n")
            f.write(f"Median Return: {percentiles[2]:.2f}%\n")
            f.write(f"95th Percentile Return: {percentiles[4]:.2f}%\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("END OF REPORT\n")
        f.write("="*60 + "\n")
    
    logger.info(f"✓ Performance metrics saved to {metrics_file}")
except Exception as e:
    logger.error(f"Failed to save performance metrics: {str(e)}")

# Save configuration for reproducibility
try:
    config_file = os.path.join(results_dir, f'synergy_3_config_{timestamp}.json')
    config_dict = {attr: getattr(StrategyConfig, attr) 
                   for attr in dir(StrategyConfig) 
                   if not attr.startswith('_') and not callable(getattr(StrategyConfig, attr))}
    
    with open(config_file, 'w') as f:
        json.dump(config_dict, f, indent=2, default=str)
    
    logger.info(f"✓ Configuration saved to {config_file}")
except Exception as e:
    logger.error(f"Failed to save configuration: {str(e)}")

logger.info("\n" + "="*60)
logger.info("NW-RQK → MLMI → FVG SYNERGY STRATEGY COMPLETE")
logger.info("="*60)

# Final summary
logger.info("\nFINAL SUMMARY:")
if portfolio is not None and stats is not None and 'Total Return [%]' in stats:
    logger.info(f"Total Return: {stats['Total Return [%]']:.2f}%")
    logger.info(f"Sharpe Ratio: {stats.get('Sharpe Ratio', 'N/A'):.2f}")
    logger.info(f"Max Drawdown: {stats.get('Max Drawdown [%]', 'N/A'):.2f}%")
    logger.info(f"Total Trades: {stats.get('Total Trades', 0)}")
else:
    logger.info("Strategy execution completed with errors - check logs for details")

logger.info(f"\nAll results saved to: {results_dir}")
logger.info("Notebook execution completed successfully!")